In [1]:
#https://files.grouplens.org/datasets/movielens/ml-25m.zip

In [1]:
import pandas as pd

In [2]:
movies=pd.read_csv("C:/Users/Admin/Desktop/movie rec system/ml-25m/movies.csv")

In [3]:
movies #build movie search engine
rating #build rec engine

NameError: name 'rating' is not defined

In [4]:
#clean all movie titles
import re
def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]","",title) #removes partentheses dashes
    

In [5]:
movies["clean_title"]=movies["title"].apply(clean_title)

In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [51]:
#convert char to nos

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer #converts titles to nos 
vectorizer=TfidfVectorizer(ngram_range=(1,2))#searches words togther
tfdif=vectorizer.fit_transform(movies["clean_title"])#transforms titles to matrix nos
tfdif


<62423x170073 sparse matrix of type '<class 'numpy.float64'>'
	with 446566 stored elements in Compressed Sparse Row format>

In [7]:
#similarity between term we enter use cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
title=input("enter title")#"Toy Story 1995" #this title is already fit since its  test data so no fit_transform
title=clean_title(title)#clean the title using clean_title fn
query=vectorizer.transform([title]) #transform title(our query) to matrix
similarity=cosine_similarity(query,tfdif).flatten()#similarity score calculated

enter title Toy Story 1995


In [8]:
similarity

array([1.        , 0.09681098, 0.06531543, ..., 0.        , 0.        ,
       0.        ])

In [9]:
#find titles that  have greatest similarity to search term
indices=np.argpartition(similarity,-5)[-5:]#finds 5 most similar titles to our query. it gives us those indexes.

In [10]:
indices

array([20497, 14813, 59767,  3021,     0], dtype=int64)

In [11]:
result=movies.iloc[indices[::-1]]#we reverse it as it returns most similar movie at the top since the movie with highest similarity score(NOT INDEX) will be at the bottom most
result

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
59767,201588,Toy Story 4 (2019),Adventure|Animation|Children|Comedy,Toy Story 4 2019
14813,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
20497,106022,Toy Story of Terror (2013),Animation|Children|Comedy,Toy Story of Terror 2013


In [12]:
import IPython
import ipywidgets as widgets
from IPython.display import display #fn we use to show diff things as output from jupyter cells
movie_input=widgets.Text(
    value="Toy Story", #default
    description="Enter Movie Title",
    disabled=False
)
movie_list=widgets.Output()
def on_type(data):#ib=nvoked whenevr we type something
    with movie_list:
        movie_list.clear_output()
        title=data["new"]#grab title from input
        if len(title)>5:
            display(search(title))
            
movie_input.observe(on_type,names='value')
display(movie_input,movie_list)

Text(value='Toy Story', description='Enter Movie Title')

Output()

In [16]:
pip install jupyter nbextension enable --py widgetsnbextension --sys-prefix
pip install jupyter labextension install --debug @jupyter-widgets/jupyterlab-manager@2.0

SyntaxError: invalid syntax (191263510.py, line 1)

In [13]:
#find movies rec similar to the movie we typed
ratings=pd.read_csv("C:/Users/Admin/Desktop/movie rec system/ml-25m/ratings.csv")

In [14]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [15]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [16]:
movieid=1#toy story 1995

In [17]:
#find all users who also liked the movie we typed and the other movies they liked
similar_users=ratings[(ratings["movieId"]==movieid)& (ratings["rating"]>4)]["userId"].unique()
similar_users #(user ids) #we are actually trying to find out the users who watched toy story and rated it >4

array([    36,     75,     86, ..., 162527, 162530, 162533], dtype=int64)

In [64]:
#now find other movies that they liked
similar_user_recs=ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"]>4)]["movieId"]
similar_user_recs#the movie ids of other 5 star movies that those same users liked 

5101            1
5105           34
5111          110
5114          150
5127          260
            ...  
24998854    60069
24998861    67997
24998876    78499
24998884    81591
24998888    88129
Name: movieId, Length: 1358326, dtype: int64

In [65]:
#movies that 10%usres or more similar to us liked
similar_user_recs.value_counts()#counts no of times each movie occurs in dataset

movieId
1         18835
318        8393
260        7605
356        6973
296        6918
          ...  
128478        1
125125        1
119701        1
107563        1
7625          1
Name: count, Length: 19282, dtype: int64

In [66]:
similar_user_recs=similar_user_recs.value_counts()/len(similar_users)
similar_user_recs #% conversion of movie id counts

movieId
1         1.000000
318       0.445607
260       0.403770
356       0.370215
296       0.367295
            ...   
128478    0.000053
125125    0.000053
119701    0.000053
107563    0.000053
7625      0.000053
Name: count, Length: 19282, dtype: float64

In [67]:
#moviids that more than 10% of users like
similar_user_recs=similar_user_recs[similar_user_recs>.1]
similar_user_recs

movieId
1        1.000000
318      0.445607
260      0.403770
356      0.370215
296      0.367295
           ...   
953      0.103053
551      0.101195
1222     0.100876
745      0.100345
48780    0.100186
Name: count, Length: 113, dtype: float64

In [68]:
#how much all the users in our dataset like these movies
#we need to find all the other users who didnot watch toy story but like movies similar to the moies people who watched toy story liked

In [69]:
all_users=ratings[(ratings["movieId"].isin(similar_user_recs.index))&(ratings["rating"]>4)]

In [70]:
all_users#we basically extract the 5 star movieids of people who watched toy story and we find the peopple outside our set who liked those movieids.

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
29,1,4973,4.5,1147869080
48,1,7361,5.0,1147880055
72,2,110,5.0,1141416589
76,2,260,5.0,1141417172
...,...,...,...,...
25000062,162541,5618,4.5,1240953299
25000065,162541,5952,5.0,1240952617
25000078,162541,7153,5.0,1240952613
25000081,162541,7361,4.5,1240953484


In [71]:
#count % of all the above users(count outside our set ofc)
all_users_recs=all_users["movieId"].value_counts()/len(all_users["userId"].unique())
all_users_recs

movieId
318      0.342220
296      0.284674
2571     0.244033
356      0.235266
593      0.225909
           ...   
551      0.040918
50872    0.039111
745      0.037031
78499    0.035131
2355     0.025091
Name: count, Length: 113, dtype: float64

In [72]:
#our set: similar users recs --> others 5 star movies that people similar to us liked
#outside our set: all users recs -->people oustide our set who liked the ABOVE 5 star movies obtained from similar users

# now we have to get movies that specifically people belonging to our set(or niche) liked and not because evrybody liked it.

# basically the similar_users_recs % must be higher than that of all_users_recs
#I KNOW U DONT UNDERSTAND SO LOOK AT EXAMPLE BELOW


#for eg: lets say toy story has 100%(or rating) in similar recs and all users recs then
#it means that people like toy story not because they are part of your set but because everybody likes it
#basically everybody likes toy story(all users and similar users)
#so its not a great recommendation to someone who likes avengers because everybody likes toy story regardless of wheether they like avengers or not

#so if theers a high %in similar_users_recs(our set) and a high % in all_users_recs then its not a good recommendation

#but lets say thor has  a 100% in similar recs (our set) and only 40% in all users recs(oustde our set). then it means that people who liked avengers
#also liked thor even though the rest of the world doesnt which is what we want! so this movie can be recommended to us. if everyone liked thor 
#then it may not be a great recommendation

# SO we have to get movies that have a high differential b/w the people who are similar to us liked it  and the people outside our set liked it.

In [73]:
#now we compare the % of all usres rec and similar uses rec(% of people liking a movie who belong to all users and % who belong to similar users)  
rec_percentages=pd.concat([similar_user_recs,all_users_recs],axis=1)
rec_percentages.columns=["similar","all"]#we are naming the coloumns as similar and all
rec_percentages#gives u the no of similar users and all users who rated a particuar movie 5 star
#we want movies that have a big diff between these 2

,similar,all
movieId,,
1,1.000000,0.124728
318,0.445607,0.342220
260,0.403770,0.222207
356,0.370215,0.235266
296,0.367295,0.284674
...,...,...
953,0.103053,0.045792
551,0.101195,0.040918
1222,0.100876,0.066877


In [74]:
#ratio between percentages
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]#adding new col score

In [75]:
rec_percentages=rec_percentages.sort_values("score",ascending=False)#sorts in descending order
#the higher the ratio the better the recommendation which is what we want. so we sort in descending to get the highest values to the top

In [76]:
rec_percentages

,similar,all,score
movieId,,,
1,1.000000,0.124728,8.017414
3114,0.280648,0.053706,5.225654
2355,0.110539,0.025091,4.405452
78499,0.152960,0.035131,4.354038
4886,0.235147,0.070811,3.320783
...,...,...,...
2858,0.216724,0.167634,1.292845
296,0.367295,0.284674,1.290232
79132,0.166817,0.131384,1.269693


In [77]:
#take top 10 recommendations and merge that with movie data to get titles of those moveis

In [79]:
rec_percentages.head(10).merge(movies,left_index=True,right_on="movieId")#use movieId from rec percentage and merge that with the movieId from the movies daatset to get the titles

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Bugs Life A 1998
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,Monsters Inc 2001
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin 1992
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Finding Nemo 2003
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast 1991
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,Incredibles The 2004
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,Lion King The 1994


In [111]:
#similarity between term we enter use cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def find_similar_movies(movieid):
    similar_users=ratings[(ratings["movieId"]==movieid)& (ratings["rating"]>4)]["userId"].unique()
    similar_user_recs=similar_user_recs.value_counts()/len(similar_users)
    
    similar_user_recs=similar_user_recs.value_counts()/len(similar_users)
    similar_user_recs=similar_user_recs[similar_user_recs>.1]
    
    all_users=ratings[(ratings["movieId"].isin(similar_user_recs.index))&(ratings["rating"]>4)]
    all_users_recs=all_users["movieId"].value_counts()/len(all_users["userId"].unique())
    
    rec_percentages=pd.concat([similar_user_recs,all_users_recs],axis=1)
    rec_percentages.columns=["similar","all"]
    
    rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
    
    rec_percentages=rec_percentages.sort_values("score",ascending=False)
    return rec_percentages.head(10).merge(movies,left_index=True,right_on="movieId")
    
    

title=input("enter title")#"Toy Story 1995" #this title is already fit since its  test data so no fit_transform
title=clean_title(title)#clean the title
movieid=movies[movies["clean_title"]==title]["movieId"].values[0]
a=find_similar_movies(movieid)
print(a)


enter title Toy Story 1995


IndexError: index 0 is out of bounds for axis 0 with size 0

In [107]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assuming you have your ratings and movies DataFrames
# ratings = ...
# movies = ...

def find_similar_movies(movieid):
    similar_users = ratings[(ratings["movieId"] == movieid) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[ratings["userId"].isin(similar_users)]["movieId"].value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > 0.1]

    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    rec_percentages_top = rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

    return rec_percentages_top

title = input("Enter title: ")
# Assuming the title is already cleaned
movieid = movies[movies["clean_title"] == title]["movieId"].values[0]
similar_movies = find_similar_movies(movieid)
print(similar_movies)


Enter title:  Toy Story 1995


ValueError: Can only compare identically-labeled Series objects

Enter title:  import pandas as pd from sklearn.metrics.pairwise import cosine_similarity import numpy as np  # Assuming you have your ratings and movies DataFrames # ratings = ... # movies = ...  def find_similar_movies(movieid):     similar_users = ratings[(ratings["movieId"] == movieid) & (ratings["rating"] > 4)]["userId"].unique()     similar_user_recs = ratings[ratings["userId"].isin(similar_users)]["movieId"].value_counts() / len(similar_users)     similar_user_recs = similar_user_recs[similar_user_recs > 0.1]      all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]     all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())      rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)     rec_percentages.columns = ["similar", "all"]      rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]      rec_percentages = rec_percentages.sort_values("score", ascending=Fa

IndexError: index 0 is out of bounds for axis 0 with size 0